In [0]:
import pandas as pd
import PIL
from PIL import Image

In [0]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16
# Load Keras' VGG16 model that was pre-trained against the ImageNet database
# model = vgg16.VGG16()
# model.summary()

In [0]:
# Path to folders with training data
images_path = Path("images")

images = []
labels = []

In [195]:
cnt_malignant = 0
cnt_benign = 0
for img,detail in zip(images_path.glob("*.jpg"),images_path.glob("*.json")):
  img = image.load_img(img,target_size=(64, 64))
  image_array = np.array(img)
  images.append(image_array)
  label = pd.read_json(detail)
  labels.append(int(label.loc['clinical','meta']['benign_malignant'] == 'malignant'))
  print(label.loc['clinical','meta']['benign_malignant'], int(label.loc['clinical','meta']['benign_malignant'] == 'malignant'))
  if(label.loc['clinical','meta']['benign_malignant']):
    cnt_malignant+=1
  else:
    cnt_benign+=1
# Create a single numpy array with all the images we loaded
x_train = np.array(images)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

benign 0
benign 0
benign 0
benign 0
benign 0
malignant 1
benign 0
benign 0
benign 0
malignant 1
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
malignant 1
benign 0
benign 0
malignant 1
benign 0
malignant 1
benign 0
malignant 1
benign 0
malignant 1
malignant 1
benign 0
benign 0
benign 0
malignant 1
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
malignant 1
malignant 1
benign 0
malignant 1
malignant 1
malignant 1
malignant 1
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
malignant 1
malignant 1
indeterminate 0
benign 0
benign 0
malignant 1
malignant 1
benign 0
malignant 1
benign 0
benign 0
benign 0
malignant 1
malignant 1
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
benign 0
malignant 1
benign 0
benign 0
benign 0
benign 0
benign 0
malignant 1
benign 0
malignant 1
benign 0
benign 0
benign 0
malignant 1
benign 0
benign 0
malignant 1
benign 0
malignant 1
benign 0
malignant 1
benign 0
malignant 1
malignant 1


['y_train.dat']

In [196]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 835, 1: 322}

In [197]:
x_train.shape

(1157, 64, 64, 3)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib

In [216]:
# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=30,
    shuffle=True
)

Epoch 1/30
1157/1157 [==============================] - 5s 4ms/step - loss: 0.6372 - acc: 0.6984
Epoch 2/30
1157/1157 [==============================] - 0s 266us/step - loss: 0.6293 - acc: 0.7156
Epoch 3/30
1157/1157 [==============================] - 0s 267us/step - loss: 0.6030 - acc: 0.7191
Epoch 4/30
1157/1157 [==============================] - 0s 267us/step - loss: 0.5980 - acc: 0.7217
Epoch 5/30
1157/1157 [==============================] - 0s 261us/step - loss: 0.5978 - acc: 0.7200
Epoch 6/30
1157/1157 [==============================] - 0s 276us/step - loss: 0.6021 - acc: 0.7208
Epoch 7/30
1157/1157 [==============================] - 0s 265us/step - loss: 0.5983 - acc: 0.7208
Epoch 8/30
1157/1157 [==============================] - 0s 274us/step - loss: 0.6010 - acc: 0.7217
Epoch 9/30
1157/1157 [==============================] - 0s 277us/step - loss: 0.5882 - acc: 0.7226
Epoch 10/30
1157/1157 [==============================] - 0s 275us/step - loss: 0.5879 - acc: 0.7200
Epoch 11/30

In [0]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")